In [3]:
import pandas as pd
import numpy as np 
import csv
import matplotlib.pyplot as plt
import math
#from pandas_datareader import data
import scipy.optimize
from scipy import stats
from datetime import datetime, timedelta

In [4]:
#calculate the number of rows in raw file
num_lines = sum(1 for line in open('DI_589_20190103.csv'))

In [5]:
#Create two pandas data frames, data for limit order info, data_MO for market order info
#Import data by each entry in every row, using dataframe.iat() function, once an entry has been successfully extract, drop this entry. 
#Drop entries without a single efficient entry.
#Drop any entry has order type doesn't equal to 0 or 1 or 2, which indicates bid, ask and market order respectively.

data=pd.DataFrame(columns=["Sequence Number","Timestamp","Update Info","Order Type","Instrument ID","Price","Volume","Price Level"])
data_MO=pd.DataFrame(columns=["Sequence Number","Execution Time","Instrument ID","Order Type","Price","Volume"])
for n in range(num_lines):    #Loop based on the number of lines in raw file
    raw_data = pd.read_csv("DI_589_20190103.csv", header=None, skiprows=n, nrows=1)    #read one raw each time, skip n rows where n start from 0
    if len(raw_data.columns)<18:    #18 is the least column number, which indicates a row contain enough columns and enough efficient data.
        continue
    else:
        if int(raw_data.iat[0,4][1:])>1:    #raw_data.iat[0,4] contains the number of entries in one row, which is in "{n" format.
            for i in range(int(raw_data.iat[0,4][1:])):
                if raw_data.iat[0,7]==2:    #raw_data.iat[0,7] contains the order type, 0: bid order, 1: offer order, 2: trade order
                    if raw_data.iat[0,17]=="{":    #raw_data.iat[0,17] contains trade type, which don't have value for trades outside the auctions
                        data_MO=data_MO.append({"Sequence Number":raw_data.iat[0,1],"Execution Time":raw_data.iat[0,14],"Order Type":raw_data.iat[0,24],"Instrument ID":raw_data.iat[0,8],"Price":raw_data.iat[0,10],"Volume":raw_data.iat[0,11]},ignore_index=True)
                        dropcols=[5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
                        raw_data=raw_data.drop(raw_data.columns[dropcols],axis=1)
                    else:
                        dropcols=[5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
                        raw_data=raw_data.drop(raw_data.columns[dropcols],axis=1)
                elif raw_data.iat[0,7]!=0 and raw_data.iat[0,7]!=1:
                    dropcols=[5,6,7,8,9,10,11,12,13,14,15,16]
                    raw_data=raw_data.drop(raw_data.columns[dropcols],axis=1)
                else:
                    data=data.append({"Sequence Number":raw_data.iat[0,1],"Timestamp":raw_data.iat[0,14],"Update Info":raw_data.iat[0,6],"Order Type":raw_data.iat[0,7],"Instrument ID":raw_data.iat[0,8],"Price":raw_data.iat[0,10],"Price Level":raw_data.iat[0,13],"Volume":raw_data.iat[0,11]},ignore_index = True)
                    dropcols=[5,6,7,8,9,10,11,12,13,14,15,16]
                    raw_data=raw_data.drop(raw_data.columns[dropcols],axis=1)
        else:
            if raw_data.iat[0,7]!=0 and raw_data.iat[0,7]!=1 and raw_data.iat[0,7]!=2:
                continue
            elif raw_data.iat[0,7]==2:
                if raw_data.iat[0,17]=="{":
                    data_MO=data_MO.append({"Sequence Number":raw_data.iat[0,1],"Execution Time":raw_data.iat[0,14],"Order Type":raw_data.iat[0,24],"Instrument ID":raw_data.iat[0,8],"Price":raw_data.iat[0,10],"Volume":raw_data.iat[0,11]},ignore_index=True)
                else:
                    continue
            else:
                data=data.append({"Sequence Number":raw_data.iat[0,1],"Timestamp":raw_data.iat[0,14],"Update Info":raw_data.iat[0,6],"Order Type":raw_data.iat[0,7],"Instrument ID":raw_data.iat[0,8],"Price":raw_data.iat[0,10],"Price Level":raw_data.iat[0,13],"Volume":raw_data.iat[0,11]},ignore_index = True)
                
#Create a list of Instrument ID which has enough efficient entries.
#Using a inefficient row rate, which is simply inefficient row number over total row number.
ineff_rows_rate = 0

Instrument_ID=[]
count=0
for i in data.duplicated(subset="Instrument ID"):
    if i == False:
        Instrument_ID.append(data.at[count,"Instrument ID"])
    count+=1
    
for i in Instrument_ID:
    ineff_rows = data.loc[(data["Instrument ID"]==i)].isnull().any(axis=1).sum()
    rows = len(data.loc[(data["Instrument ID"]==i)])
    if ineff_rows/rows >= ineff_rows_rate:
        Instrument_ID.remove(i)

Instrument_ID_MO=[]
count_MO=0
for i in data_MO.duplicated(subset="Instrument ID"):
    if i == False:
        Instrument_ID_MO.append(data_MO.at[count_MO,"Instrument ID"])
    count_MO+=1

for i in Instrument_ID_MO:
    ineff_rows_MO = data_MO.loc[(data_MO["Instrument ID"]==i)].isnull().any(axis=1).sum()
    rows_MO = len(data_MO.loc[(data_MO["Instrument ID"]==i)])
    if ineff_rows_MO/rows_MO >= ineff_rows_rate:
        Instrument_ID_MO.remove(i)

In [6]:
#Drop rows with Nan value, which indicates inefficient entry.
data=data.dropna()
data=data.reset_index(drop=True)
#data.to_csv("data_DI58920190103.csv", index = False, header=True)

In [7]:
#create bid order book sorted by each instrument.
#when update info equals to 0, which indicates new created order, the price and volume will be added to the order book.
#when update info equals to 1, which indicates updated volume of specific price, the volume related to that price will be updated.
#when update info equals to 2, which indicates drop of specific price, the relative row will be deleted.
#when update info equals to 3, which indicates drop of all price level before specific price, relative rows will be deleted.
#when update info equals to 5, which indicates updated volume and price of specific price level, change all data of that price level.
#merge all duplicates and rank them in descending order
#for each instrument_ID, export two .csv files include price and volume seperately, each of them is dynamic and changing based on nanoseconds

bid_book = {}

for i in Instrument_ID:
    bid_book[i]=pd.DataFrame(columns=["Price","Volume"])
    bid_Price_file = {}
    bid_Volume_file = {}
    instrument_data = data.loc[(data["Instrument ID"]==i) & (data["Order Type"]==0.0)]    
    for n in range(int(len(instrument_data))):
        if instrument_data.iat[n,2]==0:    #instrument_data.iat[n,2] contains the update info, which implies the new-create, change, and delete info.
            #new created order with price already exist in order book will update the volume related to existing price.
            if bid_book[i]["Price"].isin([instrument_data.iat[n,5]]).any()==True:    #instrument_data.iat[n,5] contains the price info
                ind=int(bid_book[i][bid_book[i]["Price"]==instrument_data.iat[n,5]].index.values)
                bid_book[i].iloc[ind,1]=instrument_data.iat[n,6]    #instrument_data.iat[n,6] contains the volume info
            else:
                bid_book[i].loc[len(bid_book[i].index)]=[instrument_data.iat[n,5],instrument_data.iat[n,6]]
        elif instrument_data.iat[n,2]==1:
            #update order with price doesn't exist in current order book will be added.
            if bid_book[i]["Price"].isin([instrument_data.iat[n,5]]).any()==True:
                ind=int(bid_book[i][bid_book[i]["Price"]==instrument_data.iat[n,5]].index.values)
                bid_book[i].iloc[ind,1]=instrument_data.iat[n,6]
            else:
                bid_book[i].loc[len(bid_book[i].index)]=[instrument_data.iat[n,5],instrument_data.iat[n,6]]
        elif instrument_data.iat[n,2]==2:
            #delete order with price doesn't exist in current order book will be ignored.
            if bid_book[i]["Price"].isin([instrument_data.iat[n,5]]).any()==True:
                ind=int(bid_book[i][bid_book[i]["Price"]==instrument_data.iat[n,5]].index.values)
                bid_book[i]=bid_book[i].drop([ind])
            else:
                continue
        elif instrument_data.iat[n,2]==3:
            ind=int(bid_book[i][bid_book[i]["Price"]==instrument_data.iat[n,5]].index.values)
            bid_book[i]=bid_book[i].drop(bid_book[i].index[0:ind+1])
        elif instrument_data.iat[n,2]==5:
            bid_book[i].loc[instrument_data.iat[n,7]-1]=[instrument_data.iat[n,5],instrument_data.iat[n,6]]
        bid_book[i] = bid_book[i].groupby(["Price"],as_index = False).sum()
        bid_book[i] = bid_book[i].sort_values(by=["Price"],ignore_index=True,ascending=False)        
        bid_Price_file[int(instrument_data.iat[n,1])] = list(bid_book[i]["Price"])
        bid_Volume_file[int(instrument_data.iat[n,1])] = list(bid_book[i]["Volume"])    
    bid_Price_file_name = "bid_Price_File_"+str(int(i))+".csv"
    bid_Volume_file_name = "bid_Volume_File_"+str(int(i))+".csv"
    bid_Price_file=pd.DataFrame.from_dict(bid_Price_file, orient='index')
    bid_Volume_file=pd.DataFrame.from_dict(bid_Volume_file, orient='index')
    bid_Price_file.to_csv(bid_Price_file_name, header=None)
    bid_Volume_file.to_csv(bid_Volume_file_name, header=None)


In [8]:
#create offer order book sorted by each instrument.
#when update info equals to 0, which indicates new created order, the price and volume will be added to the order book.
#when update info equals to 1, which indicates updated volume of specific price, the volume related to that price will be updated.
#when update info equals to 2, which indicates drop of specific price, the relative row will be deleted.
#when update info equals to 3, which indicates drop of all price level before specific price, relative rows will be deleted.
#when update info equals to 5, which indicates updated volume and price of specific price level, change all data of that price level.
#merge all duplicates and rank them in ascending order
#for each instrument_ID, export two .csv files include price and volume seperately, each of them is dynamic and changing based on nanoseconds

offer_book = {}

for i in Instrument_ID:
    offer_book[i]=pd.DataFrame(columns=["Price","Volume"])
    offer_Price_file = {}
    offer_Volume_file = {}
    instrument_data = data.loc[(data["Instrument ID"]==i) & (data["Order Type"]==1.0)]
    for n in range(int(len(instrument_data))):
        if instrument_data.iat[n,2]==0:
            #new created order with price already exist in order book will update the volume related to existing price.
            if offer_book[i]["Price"].isin([instrument_data.iat[n,5]]).any()==True:
                ind=int(offer_book[i][offer_book[i]["Price"]==instrument_data.iat[n,5]].index.values)
                offer_book[i].iloc[ind,1]=instrument_data.iat[n,6]
            else:
                offer_book[i].loc[len(offer_book[i].index)]=[instrument_data.iat[n,5],instrument_data.iat[n,6]]
        elif instrument_data.iat[n,2]==1:
            #update order with price doesn't exist in current order book will be added.
            if offer_book[i]["Price"].isin([instrument_data.iat[n,5]]).any()==True:
                ind=int(offer_book[i][offer_book[i]["Price"]==instrument_data.iat[n,5]].index.values)
                offer_book[i].iloc[ind,1]=instrument_data.iat[n,6]
            else:
                offer_book[i].loc[len(offer_book[i].index)]=[instrument_data.iat[n,5],instrument_data.iat[n,6]]
        elif instrument_data.iat[n,2]==2:
            #delete order with price doesn't exist in current order book will be ignored.
            if offer_book[i]["Price"].isin([instrument_data.iat[n,5]]).any()==True:
                ind=int(offer_book[i][offer_book[i]["Price"]==instrument_data.iat[n,5]].index.values)
                offer_book[i]=offer_book[i].drop([ind])
            else:
                continue
        elif instrument_data.iat[n,2]==3:
            ind=int(offer_book[i][offer_book[i]["Price"]==instrument_data.iat[n,5]].index.values)
            offer_book[i]=offer_book[i].drop(offer_book[i].index[0:ind+1])
        elif instrument_data.iat[n,2]==5:
            offer_book[i].loc[instrument_data.iat[n,7]-1]=[instrument_data.iat[n,5],instrument_data.iat[n,6]]
        offer_book[i] = offer_book[i].groupby(["Price"],as_index = False).sum()
        offer_book[i] = offer_book[i].sort_values(by=["Price"],ignore_index=True,ascending=True)        
        offer_Price_file[int(instrument_data.iat[n,1])]=list(offer_book[i]["Price"])
        offer_Volume_file[int(instrument_data.iat[n,1])]=list(offer_book[i]["Volume"])    
    offer_Price_file_name = "offer_Price_File_"+str(int(i))+".csv"
    offer_Volume_file_name = "offer_Volume_File_"+str(int(i))+".csv"
    offer_Price_file=pd.DataFrame.from_dict(offer_Price_file, orient='index')
    offer_Volume_file=pd.DataFrame.from_dict(offer_Volume_file, orient='index')
    offer_Price_file.to_csv(offer_Price_file_name, header=None)
    offer_Volume_file.to_csv(offer_Volume_file_name, header=None)

In [17]:
#extract time, order type and volume data from data_MO
#change order type from 0 and 1 to b and s respectively, where b stand for buy and s stand for sell
#for each instrument id, export a .csv file contains entire day's market order data based on nanoseconds.

for i in Instrument_ID_MO:
    instrument_data = data_MO.loc[(data_MO["Instrument ID"]==i),["Execution Time","Order Type","Volume"]]
    instrument_data = instrument_data.replace({"Order Type":0},{"Order Type":"b"})
    instrument_data = instrument_data.replace({"Order Type":1},{"Order Type":"s"})
    instrument_file_name = "MO_File_"+str(int(i))+".csv"
    instrument_data.to_csv(instrument_file_name, header=None, index=False)